# Outline

* array of acceptable genres
* Input: 10 games -->

* all movies w/in these genres, w/in accepted
* --> console as filer --> genres as filter, queried data - keywords mlo

* query: {name:keywords}

kelly will do this using postman

* data in: name, keywords

* process: calculate dist as # common keywords, to keywords in games user chose

* output: closest games to target keywords

# Code

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import KNeighborsRegressor

games = pd.read_csv("IGDB_games.csv", sep=",")
games.head()

,id,category,created_at,external_games,keywords,name,similar_games,slug,tags,updated_at,...,hypes,dlcs,bundles,expansions,expanded_games,version_parent,version_title,remakes,standalone_expansions,remasters
0,73817,0,2017-10-05 00:00:00,"[147971, 1870903]","[1097, 1219, 2396, 4260, 5570]",Spy Snatcher,"[18290, 58702, 58867, 59500, 59720, 79064, 800...",spy-snatcher,"[536872009, 536872131, 536873308, 536875172, 5...",2020-08-26 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,112119,0,2018-11-16 00:00:00,"[1616043, 1911731]",NaN,Woohoojin Bridges,NaN,woohoojin-bridges,NaN,2020-08-26 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,134130,0,2020-05-24 00:00:00,[1976473],NaN,Astrå,NaN,astra,NaN,2020-11-06 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21667,0,2016-07-28 22:40:49,"[115710, 220247, 1923649, 2006171]","[350, 934, 4134, 4144, 4428, 4841, 4964, 5315,...",Siesta Fiesta,"[20329, 25646, 28070, 36198, 55173, 55190, 559...",siesta-fiesta,"[1, 27, 268435465, 268435486, 268435487, 53687...",2021-10-05 22:26:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,172773,0,2021-10-01 17:42:35,[2127350],[19521],Legend Of Yashiyu 2,"[54775, 69007, 96217, 99118, 103303, 106987, 1...",legend-of-yashiyu-2,"[17, 27, 268435468, 536890433]",2021-10-01 18:16:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
class User:
    def __init__(self, gn, g, t):  # self, game_names, genres, themes):
        self.name = "TESTerrr"
        self.game_names = pd.DataFrame(gn, columns=['name'])
        self.genres = pd.DataFrame(g, columns=['name'])
        self.themes = pd.DataFrame(t, columns=['name'])

# '''    def printUser(self):
#         print(self.name, "'s profile:\n- ", self.game_names,
#               "\n- ", self.genres, "\n-", self.themes)'''
    # def getIds(self):



In [4]:
'''USER UPPERCASE AS FIRST LETTER'''
gn, g, t = ["Skyrim","Destiny","Mario Kart","Forza"],["Adventure","Racing","Shooter"],["Action","Survival"]
new_user = User(gn, g, t)


In [5]:
# to be getIDs
genre_key = pd.read_csv("../DataExtraction/ids_with_genres.csv", sep=",")
new_user.genres = new_user.genres.merge(genre_key, how="left", on="name")
theme_key = pd.read_csv("../DataExtraction/themes_with_ids.csv", sep=",")
new_user.themes = new_user.themes.merge(theme_key, how="left", on="name")


<h1> example with fake data </h1>

 EX:
 `{'name': ['Skyrim', 'Destiny', 'Mario Kart', 'Forza'], 'adventure': [1, 0,0,0], 'racing': [0, 0,1,1], 'shooter': [0, 1,0,0]}`


##### just use genre and themes for now, like we talked about

In [83]:
'''def removeNans(array1):
    # https://www.kite.com/python/answers/how-to-remove-nan-values-from-a-numpy-array-in-python
    nan_array = np.isnan(array1)
    not_nan_array = ~ nan_array
    array2 = array1[not_nan_array]
    return array2'''


'def removeNans(array1):\n    # https://www.kite.com/python/answers/how-to-remove-nan-values-from-a-numpy-array-in-python\n    nan_array = np.isnan(array1)\n    not_nan_array = ~ nan_array\n    array2 = array1[not_nan_array]\n    return array2'

In [84]:
games_nona = games[(~games.name.isna()) & (~games.genres.isna()) & (~games.themes.isna())]
# to match shapes
games_nona

,id,category,created_at,external_games,keywords,name,similar_games,slug,tags,updated_at,...,hypes,dlcs,bundles,expansions,expanded_games,version_parent,version_title,remakes,standalone_expansions,remasters
3,21667,0,2016-07-28 22:40:49,"[115710, 220247, 1923649, 2006171]","[350, 934, 4134, 4144, 4428, 4841, 4964, 5315,...",Siesta Fiesta,"[20329, 25646, 28070, 36198, 55173, 55190, 559...",siesta-fiesta,"[1, 27, 268435465, 268435486, 268435487, 53687...",2021-10-05 22:26:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,172773,0,2021-10-01 17:42:35,[2127350],[19521],Legend Of Yashiyu 2,"[54775, 69007, 96217, 99118, 103303, 106987, 1...",legend-of-yashiyu-2,"[17, 27, 268435468, 536890433]",2021-10-01 18:16:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008,28387,0,2017-05-02 16:09:39,"[12293, 137034, 281289, 1708343]","[464, 1026]",CABAL Online,"[28309, 30245, 34155, 35994, 55038, 55199, 103...",cabal-online,"[1, 268435460, 268435468, 536871376, 536871938]",2021-04-19 17:09:56,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11009,70300,0,2017-09-28 00:00:00,"[142023, 1874213]",[1166],"Gold, Silver, Bronze","[15851, 109606, 113149, 113895, 116352, 116353...",gold-silver-bronze,"[1, 268435470, 536872078]",2020-08-26 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# Features in our model
features = ["genres", "themes"]
 # train on games dataset
X_train_dict = games_nona[features].to_dict(orient="records")
# get features from User
X_new_dict = {
    "genres": "30", #str(new_user.genres["id"][0]), # starting with one value
    "themes": "20", #str(new_user.themes["id"][0])
}
Y_train = games_nona["name"]
Y_new = new_user.game_names

# Dummy encoding
vec = DictVectorizer(sparse=False)
vec.fit(X_train_dict)
X_train = vec.transform(X_train_dict)
X_new = vec.transform(X_new_dict)
# skip scaling
# K-Nearest Neighbors Model
model = KNeighborsRegressor(n_neighbors=3)
model.fit(X_train, Y_train) # to make x 1d?
# # Scikit-Learn expects 2-dimensional arrays, so we need to 
# # turn the Series into a DataFrame with 1 row.
# X_new = X_new.to_frame().T
'''model.predict(X_new)'''



TypeError: unsupported operand type(s) for /: 'str' and 'int'